### Udacity Data Scientist Nanodegree - Part 1 Project

For this project we are following the **CRISP-DM process**, which consists of the following parts:

1. Business Understanding 
2. Data Understanding
3. Data Preparation 
4. Modeling
5. Evaluation
6. Deployment

### 1. Business Questions

1. How does the availability varies during the year in Seattle?
2. Which are the neighbourhoods (zip codes) with better price/review ratio?
3. Can we predict the prices of the apartments?

### 2. Data Understanding

In [684]:
# Import libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [685]:
# Load files into dataframes
calendar_df = pd.read_csv('data/calendar.csv')
listings_df = pd.read_csv('data/listings.csv')
reviews_df = pd.read_csv('data/reviews.csv')

In [686]:
listings_df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


In [687]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   listing_url                       3818 non-null   object 
 2   scrape_id                         3818 non-null   int64  
 3   last_scraped                      3818 non-null   object 
 4   name                              3818 non-null   object 
 5   summary                           3641 non-null   object 
 6   space                             3249 non-null   object 
 7   description                       3818 non-null   object 
 8   experiences_offered               3818 non-null   object 
 9   neighborhood_overview             2786 non-null   object 
 10  notes                             2212 non-null   object 
 11  transit                           2884 non-null   object 
 12  thumbn

In [688]:
# Number of appartments and features
listings_df.shape

(3818, 92)

In [689]:
# List of columns wit NaN values
listings_df.isnull().sum()[listings_df.isnull().sum()>0].sort_values(ascending=False)

license                        3818
square_feet                    3721
monthly_price                  2301
security_deposit               1952
weekly_price                   1809
notes                          1606
neighborhood_overview          1032
cleaning_fee                   1030
transit                         934
host_about                      859
host_acceptance_rate            773
review_scores_accuracy          658
review_scores_checkin           658
review_scores_value             656
review_scores_location          655
review_scores_cleanliness       653
review_scores_communication     651
review_scores_rating            647
reviews_per_month               627
first_review                    627
last_review                     627
space                           569
host_response_time              523
host_response_rate              523
neighbourhood                   416
thumbnail_url                   320
medium_url                      320
xl_picture_url              

In [690]:
# We could drop the columns with more than 50% of NaNs
drop_NaNs = ['license', 'square_feet', 'monthly_price','security_deposit']

In [691]:
# Types of data
listings_df.dtypes.value_counts()

object     62
float64    17
int64      13
dtype: int64

In [692]:
# Cancellation Policies
listings_df.cancellation_policy.value_counts()

strict      1417
moderate    1251
flexible    1150
Name: cancellation_policy, dtype: int64

In [693]:
# Number of bathrooms
listings_df.bathrooms.value_counts()

1.0    2882
2.0     373
1.5     248
2.5     124
3.0      64
3.5      57
0.5      31
0.0       8
4.0       8
4.5       3
5.0       2
8.0       2
Name: bathrooms, dtype: int64

In [694]:
# Influence of number of bathrooms with review score
listings_df.groupby(['bathrooms'])['review_scores_value'].mean().sort_values(ascending=False)

bathrooms
8.0    10.000000
4.5     9.666667
2.5     9.556701
1.5     9.544554
3.5     9.540000
4.0     9.500000
0.0     9.500000
1.0     9.455339
2.0     9.389439
3.0     9.377358
5.0     9.000000
0.5     8.857143
Name: review_scores_value, dtype: float64

It seems that the number of bathrooms does not impact in the review scores (Less the number is huge!)

In [695]:
# Columns with low variability
listings_df.nunique()[listings_df.nunique()==1]

scrape_id                1
last_scraped             1
experiences_offered      1
market                   1
country_code             1
country                  1
has_availability         1
calendar_last_scraped    1
requires_license         1
jurisdiction_names       1
dtype: int64

In [696]:
# We could drop these columns
drop_low_variability = list(listings_df.nunique()[listings_df.nunique()==1].index)

In [697]:
# Columns that could be boolean
could_boolean = list(listings_df.nunique()[listings_df.nunique()==2].index)

In [698]:
listings_df[could_boolean].dtypes

host_acceptance_rate                object
host_is_superhost                   object
host_has_profile_pic                object
host_identity_verified              object
state                               object
is_location_exact                   object
instant_bookable                    object
require_guest_profile_picture       object
require_guest_phone_verification    object
dtype: object

No one of these columns are boolean, so this is something we should treat in the next chapter

In [699]:
price_columns = [column for column in listings_df.columns if 'price' in column]
price_columns

['price', 'weekly_price', 'monthly_price']

In [700]:
host_columns = [column for column in listings_df.columns if 'host' in column]
host_columns

['host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'calculated_host_listings_count']

In [701]:
review_columns = [column for column in listings_df.columns if 'review' in column]
review_columns

['number_of_reviews',
 'first_review',
 'last_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'reviews_per_month']

In [702]:
calendar_df.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [703]:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   listing_id  1393570 non-null  int64 
 1   date        1393570 non-null  object
 2   available   1393570 non-null  object
 3   price       934542 non-null   object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB


In [704]:
calendar_df.isnull().sum()

listing_id         0
date               0
available          0
price         459028
dtype: int64

In [705]:
calendar_df.shape

(1393570, 4)

Let's compare the price values for a listing between the calendar and listing dataframes

In [706]:
listings_df.id[1], listings_df.price[1]

(953595, '$150.00')

In [707]:
calendar_df[calendar_df.listing_id == listings_df.id[1]].dropna(subset=['price'])['price'].value_counts()

$125.00    37
$144.00     7
$158.00     7
$190.00     7
$186.00     6
           ..
$141.00     1
$197.00     1
$294.00     1
$159.00     1
$400.00     1
Name: price, Length: 92, dtype: int64

For some cases the price is fix and the value is the same in both df, in other cases the price may vary. When we clean the data, we will adress it and see if the mean is taken for the listing dataframe

** Data Preparation needed for Listings DF**

- select which columns use
- differenciate cat/num columms
- check if types
- decide which drop
- clean columns and fillna

**Data Preparation needed Calendar DF**

- date to datetime
- available boolean -> True/False -> Get dummies
- price should be float
- Merge with listings_df in new df

### 3. Data Preparation

In [708]:
# Calendar_df

# Convert date to correct format
calendar_df.date = pd.to_datetime(calendar_df.date)

# Convert price NaNs to "0" and object to float 
calendar_df.price = calendar_df.price.str.replace('$','').str.replace(',','')
calendar_df.price.fillna(0, inplace=True)
calendar_df.price = calendar_df.price.astype('float64')

# Set availability to boolean (0,1)
def map_boolean(dfs):
    mapping = {'t':1,'f':0}
    dfs = dfs.map(mapping).astype(np.uint8)
    return dfs

calendar_df.available = map_boolean(calendar_df.available)

In [709]:
# Listings_df
df_l = listings_df.copy()

# Drop previous selected columns
df_l.drop(drop_low_variability,axis=1, inplace=True)
df_l.drop(drop_NaNs,axis=1, inplace=True)

# Select only necessary columns
selected = ['id','neighbourhood','zipcode','smart_location', 'latitude', 'longitude', 'is_location_exact',
       'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'bed_type', 'amenities', 'price', 'weekly_price',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights','availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews','review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count', 'reviews_per_month']

df_l = df_l[selected]

# Change f/t to 0/1
bool_col = list(df_l.nunique()[df_l.nunique()==2].index)
for col in bool_col:
    df_l[col] = map_boolean(df_l[col])

# price,weekly_price,cleaning_fee,extra_people should be floats
for col in ['price','weekly_price','cleaning_fee','extra_people']:
    df_l[col] = df_l[col].str.replace('$','').str.replace(',','')
    df_l[col] = df_l[col].astype('float64')
    df_l[col].fillna(df_l[col].mean(), inplace=True)
    

# Select continous and categorical columns
cont_cols = df_l.select_dtypes(exclude=['object'])
cat_cols = df_l.select_dtypes(include=['object'])

In [711]:
# Continous columns with missing values
cont_cols.isnull().sum()[cont_cols.isnull().sum()>0]

bathrooms                       16
bedrooms                         6
beds                             1
review_scores_rating           647
review_scores_accuracy         658
review_scores_cleanliness      653
review_scores_checkin          658
review_scores_communication    651
review_scores_location         655
review_scores_value            656
reviews_per_month              627
dtype: int64

In [712]:
# Fill missing values with the mean
for  col in cont_cols.columns:
    cont_cols[col].fillna(cont_cols[col].mean(),inplace=True)

C:\Users\Guillermo\Anaconda3\lib\site-packages\pandas\core\series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [713]:
# How manu different values are for the catergorical columns
cat_cols.nunique()

neighbourhood            81
zipcode                  28
smart_location            7
property_type            16
room_type                 3
bed_type                  5
amenities              3284
cancellation_policy       3
dtype: int64

We see here that amenities has too many different values. This could be a dimensionality problem for get dummies. Maybe we could just count how manu amenities is a listing offering and treat the column as continous one.

In [714]:
cat_cols.amenities = [len(amenitie.split(',')) for amenitie in cat_cols.amenities]

C:\Users\Guillermo\Anaconda3\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [715]:
cont_cols = pd.concat([cont_cols, cat_cols.amenities],axis=1)

In [716]:
cat_cols.drop(['amenities'],axis=1,inplace=True)

C:\Users\Guillermo\Anaconda3\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [717]:
# Categorical columns with missing values
cat_cols.isnull().sum()[cat_cols.isnull().sum()>0]

neighbourhood    416
zipcode            7
property_type      1
dtype: int64

We can't use the categorical columns directly, so we have to preprocess them with get_dummies:

In [718]:
for col in cat_cols.columns:
    cat_cols = pd.concat([cat_cols.drop([col],axis=1),pd.get_dummies(cat_cols[col], prefix=col, prefix_sep = '_',dummy_na=True)],axis=1)

In [719]:
cat_cols

,neighbourhood_Alki,neighbourhood_Arbor Heights,neighbourhood_Atlantic,neighbourhood_Ballard,neighbourhood_Belltown,neighbourhood_Bitter Lake,neighbourhood_Brighton,neighbourhood_Broadview,neighbourhood_Broadway,neighbourhood_Bryant,...,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,bed_type_nan,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_nan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3814,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3815,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3816,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [720]:
# Concat cat/cont dateframes in one new cleaned dataframe:
df_l_cleaned = pd.concat([cont_cols,cat_cols],axis=1)

In [721]:
# Check if there are still NaNs values
df_l_cleaned.isnull().sum()[df_l_cleaned.isnull().sum()>0]

Series([], dtype: int64)

In [722]:
# Check the columns types
df_l_cleaned.select_dtypes(include=['object'])

""
0
1
2
3
4
...
3813
3814
3815
3816


In [723]:
df_l_cleaned.shape

(3818, 183)

In [724]:
# Let's check now if the prices in the listing dateframe are the mean of the calendar dataframe

df_l_cleaned.id[1], df_l_cleaned.price[1]

(953595, 150.0)

In [725]:
tmp = calendar_df[calendar_df.price>0]

In [726]:
tmp[tmp.listing_id == df_l_cleaned.id[1]].price.mean(),tmp[tmp.listing_id == df_l_cleaned.id[1]].price.mode()

(170.93127147766322,
 0    125.0
 dtype: float64)

It is nor the mean nor the mode, for the prediction purpuses we will at first use the price value of the listing dataframe

### We can answer now our first two business questions:

**1. How does the availability varies during the year in Seattle?**

In [727]:
mean_prices = calendar_df.groupby(['date']).price.mean()

In [728]:
mean_prices = mean_prices.reset_index()

In [729]:
prices = mean_prices.price.values
type(prices)
dates = np.array(range(0,len(mean_prices.price)))
type(dates)

numpy.ndarray

In [730]:
calendar_df.rename(columns = {'listing_id':'id'},inplace=True)
calendar_df.columns

Index(['id', 'date', 'available', 'price'], dtype='object')

In [731]:
df = calendar_df.merge(df_l[['id','zipcode']], on = 'id')

In [732]:
df.zipcode.nunique()

28

In [733]:
calendar_df['year']= calendar_df['date'].dt.year
calendar_df['month']= calendar_df['date'].dt.month
calendar_df['day']= calendar_df['date'].dt.day
calendar_df['week_of_year'] = calendar_df['date'].dt.week
calendar_df['day_of_week'] = calendar_df['date'].dt.dayofweek

ipykernel_launcher:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [734]:
calendar_df

,id,date,available,price,year,month,day,week_of_year,day_of_week
0,241032,2016-01-04,1,85.0,2016,1,4,1,0
1,241032,2016-01-05,1,85.0,2016,1,5,1,1
2,241032,2016-01-06,0,0.0,2016,1,6,1,2
3,241032,2016-01-07,0,0.0,2016,1,7,1,3
4,241032,2016-01-08,0,0.0,2016,1,8,1,4
...,...,...,...,...,...,...,...,...,...
1393565,10208623,2016-12-29,0,0.0,2016,12,29,52,3
1393566,10208623,2016-12-30,0,0.0,2016,12,30,52,4
1393567,10208623,2016-12-31,0,0.0,2016,12,31,52,5
1393568,10208623,2017-01-01,0,0.0,2017,1,1,52,6


In [735]:
calendar_df[calendar_df.available==1].groupby(['month']).price.mean().sort_values(ascending=False)

month
7     152.094150
8     150.656594
6     147.473137
9     143.255949
5     139.538183
12    137.251835
10    137.031939
11    135.688738
4     135.097005
3     128.644488
2     124.293927
1     122.912176
Name: price, dtype: float64

In [736]:
calendar_df[calendar_df.available==1].groupby(['month']).price.describe()

,count,mean,std,min,25%,50%,75%,max
month,,,,,,,,
1,64937.0,122.912176,88.708143,12.0,70.0,99.0,148.0,1000.0
2,73321.0,124.293927,91.550076,20.0,72.0,100.0,149.0,1000.0
3,83938.0,128.644488,94.371410,20.0,75.0,100.0,150.0,1099.0
4,76037.0,135.097005,98.232098,20.0,75.0,110.0,160.0,1099.0
5,79971.0,139.538183,102.863280,20.0,75.0,110.0,169.0,1099.0
6,77244.0,147.473137,112.481803,20.0,76.0,110.0,175.0,1099.0
7,74222.0,152.094150,120.078098,20.0,75.0,115.0,180.0,1240.0
8,76347.0,150.656594,118.754189,20.0,75.0,114.0,179.0,1250.0
9,77246.0,143.255949,108.804290,20.0,75.0,110.0,175.0,1450.0


In [737]:
calendar_df.groupby(['day_of_week']).price.mean().sort_values(ascending=False)

day_of_week
4    95.678612
5    95.624370
6    91.837727
3    91.403801
0    91.052596
2    90.992873
1    90.988420
Name: price, dtype: float64

### 4. Modeling

In [738]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Separate the dependent and independent variables and drop variables that we dont need
X = df_l_cleaned.drop(['price','id','latitude','longitude'],axis=1)
y = df_l_cleaned.price

# Split the data in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, shuffle=True)

In [739]:
X

,is_location_exact,accommodates,bathrooms,bedrooms,beds,weekly_price,cleaning_fee,guests_included,extra_people,minimum_nights,...,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,bed_type_nan,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_nan
0,1,4,1.0,1.0,1.0,788.481832,61.710904,2,5.0,1,...,0,0,0,0,1,0,0,1,0,0
1,1,4,1.0,1.0,1.0,1000.000000,40.000000,1,0.0,2,...,0,0,0,0,1,0,0,0,1,0
2,1,11,4.5,5.0,7.0,788.481832,300.000000,10,25.0,4,...,0,0,0,0,1,0,0,0,1,0
3,1,3,1.0,0.0,2.0,650.000000,61.710904,1,0.0,1,...,0,0,0,0,1,0,1,0,0,0
4,1,6,2.0,3.0,3.0,788.481832,125.000000,6,15.0,1,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,1,6,2.0,3.0,3.0,788.481832,230.000000,1,0.0,3,...,0,0,0,0,1,0,0,0,1,0
3814,1,4,1.0,1.0,2.0,788.481832,50.000000,3,25.0,2,...,0,0,0,0,1,0,0,1,0,0
3815,0,2,1.0,1.0,1.0,450.000000,35.000000,2,20.0,1,...,0,0,0,0,1,0,0,1,0,0
3816,0,2,1.0,0.0,1.0,788.481832,45.000000,1,0.0,3,...,0,0,0,0,1,0,0,1,0,0


In [743]:
df_l_cleaned.price.describe()

count    3818.000000
mean      127.976166
std        90.250022
min        20.000000
25%        75.000000
50%       100.000000
75%       150.000000
max      1000.000000
Name: price, dtype: float64

In [758]:
# Train the model
reg = LinearRegression()
reg.fit(X_train, y_train)

# Calculate predictions for the test set
preds = reg.predict(X_test)

# Evaluate the test set
print(mean_squared_error(y_test,preds,squared=False)), print(r2_score(y_test,preds))

48.136841949129376
0.6799178115029063


(None, None)